# 02 — Data Understanding (CRISP-DM)

Nesta etapa do CRISP-DM, o objetivo é **entender profundamente os dados disponíveis**,
avaliando sua estrutura, qualidade, volume e possíveis limitações.

Não há, neste momento, qualquer tipo de tratamento ou limpeza dos dados.
O foco é responder perguntas como:
- Quais dados estão disponíveis?
- Como eles estão estruturados?
- Existem valores ausentes?
- Há dados sensíveis?
- O que pode impactar as análises futuras?

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

In [4]:
from pathlib import Path

DATA_RAW_PATH = Path("../data/raw")

csv_files = sorted(DATA_RAW_PATH.glob("*.csv"))

len(csv_files), csv_files

(12,
 [WindowsPath('../data/raw/abr2025.csv'),
  WindowsPath('../data/raw/ago2025.csv'),
  WindowsPath('../data/raw/dez2025.csv'),
  WindowsPath('../data/raw/fev2025.csv'),
  WindowsPath('../data/raw/jan2025.csv'),
  WindowsPath('../data/raw/jul2025.csv'),
  WindowsPath('../data/raw/jun2025.csv'),
  WindowsPath('../data/raw/mai2025.csv'),
  WindowsPath('../data/raw/mar2025.csv'),
  WindowsPath('../data/raw/nov2025.csv'),
  WindowsPath('../data/raw/out2025.csv'),
  WindowsPath('../data/raw/set2025.csv')])

In [8]:
dfs = []

for file in csv_files:
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df["arquivo_origem"] = file.name
    dfs.append(df)

df_raw = pd.concat(dfs, ignore_index=True)

df_raw.shape

(24009, 16)

In [9]:
df_raw.head()

,Referência,Nome,Cargo,Data Admissão,Tipo de Regime,Descontos,Liquido,Data Desligamento,Proventos,Contrato,Atividade,Nome Atividade,Tipo de Contrato,Data Prevista Termino Contrato,Carga Horária (Sem.),arquivo_origem
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,40.00,abr2025.csv
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.00,NaN,NaN,Efetivo,NaN,40.00,abr2025.csv
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,30.00,abr2025.csv
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.00,NaN,NaN,Efetivo,NaN,30.00,abr2025.csv
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,40.00,abr2025.csv


## Observações sobre codificação dos arquivos

Durante a ingestão dos dados, foi identificado que os arquivos CSV
não utilizam codificação UTF-8, resultando em erros de leitura.

Após análise, foi adotada a codificação `latin1`, comum em dados
oriundos de sistemas legados da administração pública brasileira,
permitindo a correta leitura de caracteres acentuados.


In [10]:
schemas = {
    file.name: set(pd.read_csv(file, encoding="latin1", sep=";", nrows=0).columns)
    for file in csv_files
}

schemas

{'abr2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'Data Prevista Termino Contrato',
  'Descontos',
  'Liquido',
  'Nome',
  'Nome Atividade',
  'Proventos',
  'Referência',
  'Tipo de Contrato',
  'Tipo de Regime'},
 'ago2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'Data Prevista Termino Contrato',
  'Descontos',
  'Liquido',
  'Nome',
  'Nome Atividade',
  'Proventos',
  'Referência',
  'Tipo de Contrato',
  'Tipo de Regime'},
 'dez2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'Data Prevista Termino Contrato',
  'Descontos',
  'Liquido',
  'Nome',
  'Nome Atividade',
  'Proventos',
  'Referência',
  'Tipo de Contrato',
  'Tipo de Regime'},
 'fev2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'D

In [11]:
unique_schemas = set(tuple(sorted(cols)) for cols in schemas.values())
len(unique_schemas)

1

## Resumo da compreensão dos dados

- Os dados estão distribuídos em múltiplos arquivos CSV, um por período.
- Todos os arquivos apresentam **schema consistente**, permitindo concatenação segura.
- A codificação utilizada é `latin1`, compatível com sistemas legados.
- Não foram identificadas divergências estruturais entre os meses.
- Existem colunas com dados potencialmente sensíveis, que serão tratadas
  nas próximas etapas.
